In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "./Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 데이터를 임베딩으로 변환
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)
# FAISS 벡터스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

In [7]:
# 저장한 FAISS DB 불러오기
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
persist_directory = "./DB"

vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [8]:
from pydantic import Field
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from sentence_transformers import CrossEncoder
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

c:\Users\User\anaconda3\envs\MCP-Project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# ms-marco-MiniLM-L-12-v2 모델 다운로드
crossencoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

c:\Users\User\anaconda3\envs\MCP-Project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
class Retriever_with_cross_encoder(BaseRetriever):
    vectorstore: Any = Field(description="초기 검색을 위한 벡터 저장소")
    crossencoder: Any = Field(description="재순위화를 위한 크로스 인코더 모델")
    k: int = Field(default=5, description="초기에 검색할 문서 수")
    rerank_top_k: int = Field(default=2, description="재순위화 후 최종적으로 반환할 문서 수")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # 초기 검색
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)

        # 인코더용 쌍 준비
        pairs = [[query, doc.page_content] for doc in initial_docs]

        # 인코더 점수 획득
        scores = self.crossencoder.predict(pairs)

        # 점수별 문서 정렬
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)

        # 상위 재순위화 문서 반환
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

C:\Users\User\AppData\Local\Temp\ipykernel_23904\3368951697.py:1: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class Retriever_with_cross_encoder(BaseRetriever):


In [11]:
# 크로스인코더 기반 리트리버 인스턴스 생성
cross_encoder_retriever = Retriever_with_cross_encoder(
    vectorstore=vectordb,
    crossencoder=crossencoder,
    k=4,  # 초기 밀집검색으로 반환할 문서 수를 설정
    rerank_top_k=2  # 리랭킹을 통해 최종적으로 반환할 문서 수를 설정
)

# 답변용 LLM 인스턴스 생성
llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o")

In [12]:
# RetrievalQA 체인 인스턴스 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

In [13]:
query = "이 회사의 2022년 영업손실이 정확히 얼마야?"
result = qa_chain({"query": query})

print(f"\n질문: {query}")
print(f"답변: {result['result']}")
print("\n답변 근거 문서:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print each document

C:\Users\User\AppData\Local\Temp\ipykernel_23904\802996342.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



질문: 이 회사의 2022년 영업손실이 정확히 얼마야?
답변: 이 회사의 2022년 영업손실은 149.1억원입니다.

답변 근거 문서:

Document 1:
이어, 당사는 2022년 GMP시설의 위탁생산계약 매출 약 4.8억원이 발생하였으나, 종업원급
여 43.9억원, 유무형자산상각비 25.3억원이 발생하였으며, 연구개발 및 임상시험 지속 과정
에서 지급수수료가 54.8억원 발생하는 등 총 영업비용 154억원이 발생하였습니다. 이에
2022년 영업손실 149억원이 발생하였으며, 한편, 2022년 5월 금융감독원에서 공표한 '전환
사채 콜옵션 회계처리' 감독지침에 따라 당사는 2021년 3월 19일 발행한 제2회 전환사채의

Document 2:
하여 2021년 영업손실 130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원, 2024년
1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면에서도, 금융비용 등의 발생 영향
으로 인해 2021년 당기순손실 130.7억원, 2022년 당기순손실 228.7억원, 2023년 당기순손실
116.1억원, 2024년 1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면하지 못하고
있습니다.따라서 당사의 파이프라인에서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루
